# Automatic Card Generation from Template 
## *Image Manipulation using Python*

---

```bash
pip install pillow
```

In [11]:
from flask import Flask, render_template, request, send_file
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import io

app = Flask(__name__)

# Load data and font
df = pd.read_csv("students.csv")
records = df.to_dict(orient='records')
font = ImageFont.truetype("OpenSans-Semibold.ttf", size=25)

@app.route('/')
def index():
    return '''
    <h1>Student ID Card Generator</h1>
    <form action="/generate" method="post">
        <label for="student_id">Enter Student ID:</label>
        <input type="text" id="student_id" name="student_id">
        <input type="submit" value="Generate ID Card">
    </form>
    '''

@app.route('/generate', methods=['POST'])
def generate_card():
    student_id = request.form['student_id']
    
    # Find student data
    student_data = next((item for item in records if str(item['id']) == student_id), None)
    
    if not student_data:
        return "<h2>Student not found!</h2>"
    
    # Generate ID Card
    template = Image.open("templates/template.png")
    photo_path = f"photos/{student_data['id']}.jpg"
    
    if not os.path.exists(photo_path):
        return "<h2>Photo not found!</h2>"
    
    pic = Image.open(photo_path).resize((165, 190), Image.Resampling.LANCZOS)
    template.paste(pic, (25, 75))
    
    draw = ImageDraw.Draw(template)
    draw.text((315, 80), str(student_data['id']), font=font, fill='black')
    draw.text((315, 125), student_data['name'], font=font, fill='black')
    draw.text((315, 170), student_data['class'], font=font, fill='black')
    draw.text((315, 215), student_data['dob'], font=font, fill='black')
    
    # Save to memory and send as response
    img_io = io.BytesIO()
    template.save(img_io, 'JPEG')
    img_io.seek(0)
    
    return send_file(img_io, mimetype='image/jpeg', as_attachment=True, download_name=f"{student_data['id']}_card.jpg")

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
